In [1]:
import numpy as np
import keras

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras import optimizers
from PIL import Image
from keras import models
from keras import layers
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Activation
from keras.applications.vgg16 import preprocess_input
from keras.activations import linear
from keras.optimizers import SGD
from keras.datasets import cifar10
from keras.initializers import he_normal

import matplotlib.pylab as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
(train_data,train_labels),(test_data,test_labels) = cifar10.load_data()
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_data = train_data/255
test_data = test_data/255
train_labels = keras.utils.to_categorical(train_labels.astype('float32'))
test_labels = keras.utils.to_categorical(test_labels.astype('float32'))

print(train_data.shape)

170500096/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3)


In [3]:
from keras.applications import VGG16

vgg = VGG16(weights = 'imagenet',
                 include_top = False,
                 input_shape = (32, 32, 3))
vgg.summary()





58892288/58889256 [==============================] - 2s 0us/step






Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
____________________________________________________

In [0]:
# Squeeze and Excitation
def se_block(input, channels, r=8):
    # Squeeze
    x = layers.GlobalAveragePooling2D()(input)
    # Excitation
    x = layers.Dense(channels//r, activation="relu")(x)
    x = layers.Dense(channels, activation="sigmoid")(x)
    return layers.Multiply()([input, x])

In [5]:
x = vgg.layers[0].input
for i, layer in enumerate(vgg.layers):
    if i == 0: continue
    if "conv" in layer.name:
        layer.activation = linear
        x = layer(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = se_block(x, layer.filters)
    else:
        x = layer(x)

conv_base = Model(inputs=vgg.layers[0].input, outputs=x)

In [6]:
model = models.Sequential()
model.add(conv_base)

model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(256, activation='relu', kernel_initializer=he_normal()))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 1, 1, 512)         15188944  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 256)               131328    
_________________________________________________________________
batch_normalization_15 (Batc (None, 256)               1024      
____

In [0]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False    

In [8]:
from keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=1e-5, decay=1e-6),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [0]:
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint, ReduceLROnPlateau

es_cb = EarlyStopping(monitor='val_loss', 
                              patience=10, 
                              verbose=1, 
                              mode='auto')

mc_cb = ModelCheckpoint("model_ck.h5", 
                        monitor='val_loss', 
                        verbose=0,
                        save_best_only=False, 
                        save_weights_only=False, 
                        mode='auto', 
                        period=1)

lr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1
)

In [10]:
history = model.fit(train_data, train_labels,
                    batch_size=32,
                    validation_data=(test_data, test_labels),
                    # callbacks=[es_cb, mc_cb, lr_cb],
                    epochs = 10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 42s 833us/step - loss: 1.8947 - acc: 0.4353 - val_loss: 1.0051 - val_acc: 0.6578
Epoch 2/10
50000/50000 [==============================] - 36s 718us/step - loss: 1.3399 - acc: 0.5851 - val_loss: 0.8774 - val_acc: 0.7012
Epoch 3/10
50000/50000 [==============================] - 36s 714us/step - loss: 1.1702 - acc: 0.6317 - val_loss: 0.8175 - val_acc: 0.7245
Epoch 4/10
50000/50000 [==============================] - 36s 711us/step - loss: 1.0680 - acc: 0.6602 - val_loss: 0.7793 - val_acc: 0.7360
Epoch 5/10
50000/50000 [==============================] - 36s 712us/step - loss: 1.0035 - acc: 0.6766 - val_loss: 0.7564 - val_acc: 0.7479
Epoch 6/10
50000/50000 [==============================] - 36s 711us/step - loss: 0.9487 - acc: 0.6920 - val_loss: 0.7369 - val_acc: 0.7527
Epoch 7/10
50000/500

In [0]:
# 精度の履歴を可視化する。
ha = history.history['acc']
va = history.history['val_acc']
epochs = range(len(ha))
plt.plot(epochs, ha, 'ro', label='T acc')
plt.plot(epochs, va, 'g', label='V acc')
plt.legend();

In [0]:
# 損失関数の履歴を可視化する。
ha = history.history['loss']
va = history.history['val_loss']
epochs = range(len(ha))
plt.plot(epochs, ha, 'ro', label='T loss')
plt.plot(epochs, va, 'g', label='V loss')
plt.legend();

In [0]:
# 評価
test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f"test loss: {test_loss:.2f}, test accuracy: {test_acc:.2%}")